In [7]:
import os
import requests
from typing import Type
from pydantic import BaseModel, Field
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, AgentType
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
from langchain.schema import SystemMessage
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize the LLM
llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo")

# Define the tools
class WikipediaSearchTool(BaseTool):
    name = "WikipediaSearchTool"
    description = "Search Wikipedia for a specific query."

    class Args(BaseModel):
        query: str = Field(description="The query to search on Wikipedia.")

    def _run(self, query: str):
        ddg = DuckDuckGoSearchAPIWrapper()
        results = ddg.run(f"site:wikipedia.org {query}")
        return results

class DuckDuckGoSearchTool(BaseTool):
    name = "DuckDuckGoSearchTool"
    description = "Search the web using DuckDuckGo."

    class Args(BaseModel):
        query: str = Field(description="The query to search on DuckDuckGo.")

    def _run(self, query: str):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)

class WebScraperTool(BaseTool):
    name = "WebScraperTool"
    description = "Scrape text content from a given URL."

    class Args(BaseModel):
        url: str = Field(description="The URL of the website to scrape.")

    def _run(self, url: str):
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            return f"Failed to fetch the URL. Status code: {response.status_code}"

class SaveToFileTool(BaseTool):
    name = "SaveToFileTool"
    description = "Save the given content to a text file."

    class Args(BaseModel):
        filename: str = Field(description="The name of the file to save the content.")
        content: str = Field(description="The content to save into the file.")

    def _run(self, filename: str, content: str):
        with open(filename, "w", encoding="utf-8") as file:
            file.write(content)
        return f"Content saved to {filename}"

# Initialize the agent
agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WikipediaSearchTool(),
        DuckDuckGoSearchTool(),
        WebScraperTool(),
        SaveToFileTool(),
    ],
    agent_kwargs={
        "system_message": SystemMessage(
            content="""
            You are a research assistant AI agent. Your task is to research queries using Wikipedia and DuckDuckGo, 
            extract relevant information from websites, and save your findings to a text file. Always return a 
            clear summary of your actions and results.
            """
        )
    },
)

In [8]:
# Test the agent with a query
query = "Research about the XZ backdoor"
response = agent.invoke(query)
print(response)



> Entering new AgentExecutor chain...

Invoking: `WikipediaSearchTool` with `XZ backdoor`


Main page; Contents; Current events; Random article; About Wikipedia; Contact us; Donate; Pages for logged out editors learn more Main page; Contents; Current events; Random article; About Wikipedia; Contact us; Donate Discussion: Attaque de XZ Utils par porte dérobée. Ajouter des langues. Le contenu de la page n'est pas pris en charge dans d'autres langues. Article; Discussion; français. Lire; Modifier le code; Ajouter un sujet; Voir l'historique; Outils. Outils. déplacer vers la barre latérale masquer. Actions Lire; Main page; Contents; Current events; Random article; About Wikipedia; Contact us; Donate; Pages for logged out editors learn more Main page; Contents; Current events; Random article; About Wikipedia; Contact us; Donate; Pages for logged out editors learn more
Invoking: `DuckDuckGoSearchTool` with `XZ backdoor`
responded: I couldn't find specific information about the "XZ backdoor

In [9]:
import json

# Save the response to a file
save_tool = SaveToFileTool()
filename = "research_results.txt"

# Convert the response dict to a JSON string
content = json.dumps(response, indent=4)

# Prepare tool_input as a dictionary
tool_input = {
    "filename": filename,
    "content": content
}

# Call the run method with tool_input
save_message = save_tool.run(tool_input=tool_input)
print(save_message)

Content saved to research_results.txt
